In [3]:
from calculate_colocations import get_pos_filtered_colloc_from_corpus_list, get_colloc_from_corpus_list,get_clean_lemm_list
import pandas as pd
from tqdm import tqdm
import json

In [2]:
texts_lenta = pd.read_csv("3000.csv")

texts_lenta.iloc[0]['texts_3000']

'Указом президента России Бориса Ельцина внесены изменения в  существующую структуру Федеральной службы безопасности РФ, утвержденную в июле прошлого года. Как говорится в поступившем сегодня сообщении Центра общественных связей ФСБ, в соответствии с Основами (Концепцией) государственной политики Российской Федерации по военномустроительству на период до 2005 года, на базе Департамента по борьбе с терроризмом и Управления конституционной безопасности ФСБ создан Департамент по защите конституционного строя и борьбе с терроризмом. В составе департамента организуются три управления с четко определенной компетенцией. В ФСБ отмечают, что "в современных условиях для российскойгосударственности имеют приоритетное значение вопросы защитыконституционного строя, сохранения целостности страны, борьбыс терроризмом и всеми формами экстремизма, а также разведывательно-подрывной деятельностью спецслужб и организаций иностранных государств". Как подчеркивается в сообщении, "органам безопасности в реше

In [6]:
clean_lemm_list = get_clean_lemm_list(list(texts_lenta['texts_3000']),lang = 'rus')

100%|██████████| 898/898 [02:18<00:00,  6.47it/s]


In [8]:
from collections import Counter
z = ['blue', 'red', 'blue', 'yellow', 'blue', 'red']
Counter(z)


Counter({'blue': 3, 'red': 2, 'yellow': 1})

In [10]:
unigramm_freq = Counter(clean_lemm_list)

In [11]:
'президент' in unigramm_freq

True

In [13]:
with open ("unigr_freq.json","w") as f:
    json.dump(unigramm_freq, f, indent = 4, ensure_ascii = False)

In [ ]:

bigramFreqTable, trigramFreqTable, quadgram_freq, filtered_bi, filtered_tri, bigramPMITable, trigramPMITable, quadragramPMITable, bigramChiTable, trigramChiTable =get_pos_filtered_colloc_from_corpus_list(list(texts_lenta['texts_3000']),"rus")


In [ ]:
def cean_pos_tags(df, ngramm_name):
    clean_words = []
    for i in tqdm(range(len(df))):
        posed_ngramm = df.iloc[i][ngramm_name]
        clean_ngramm = []
        for w in posed_ngramm:
            word = w.split("_")[0]
            clean_ngramm.append(word)
        clean_ngramm = tuple(clean_ngramm)
        clean_words.append(clean_ngramm)
    
    data = { "ngramm":clean_words,"freq":df['freq']}
    clean_df = pd.DataFrame(data) 
    return clean_df
bigramFreqTable_clean = cean_pos_tags(bigramFreqTable,'bigram')
trigramFreqTable_clean = cean_pos_tags(trigramFreqTable, "trigram")
quadgram_freq_clean = cean_pos_tags(quadgram_freq, "quadgramF")

In [ ]:
filtered_tri.head()

In [ ]:
filtered_bi_clean = cean_pos_tags(filtered_bi,'bigram')
filtered_tri_clean = cean_pos_tags(filtered_tri,'trigram')

In [ ]:
filtered_bi_clean.head()

In [ ]:
def get_freq_colloc_dict(ngrm_lis):
    freq_colloc_dict = {'2':{},'3':{},'4':{}}
    for ngramm_df in ngrm_lis:
        dct = ngramm_df.to_dict("split")
        words_len = len(dct['data'][0][0])
        for el in dct['data']:
            
            ngramm_raw = ''
            for el_i in el[0]:
                ngramm_raw += el_i + ' '
            ngramm_raw = ngramm_raw.strip()
            #print(ngramm_raw,words_len )
            freq_colloc_dict[str(words_len)][ngramm_raw] = el[1]
        
    return freq_colloc_dict
ngramms_list =[bigramFreqTable_clean, trigramFreqTable_clean, quadgram_freq_clean]
freq_colloc_dict = get_freq_colloc_dict(ngramms_list)

In [ ]:
def get_united_numeric_colloc_base (numeric_calc_collocations_list, collocations_by_freq_dict):#все кроме фильтрованных по частям речи
    
    overall_colloc_json = {'2':{},'3':{},'4':{}}
    for colloc in numeric_calc_collocations_list:
        colloc_len = len(colloc)
        for key in list(colloc.keys()):
            if 'gram' in key:
                ngramm_name = key
            else:
                freq_name = key
        n_of_words = str(len(colloc.iloc[0][ngramm_name]))
        print(n_of_words)
        for index in tqdm(range(int(colloc_len))):
            collocation_element = colloc.iloc[index][ngramm_name]
            str_el = ''
            for el in collocation_element:
                str_el += el + ' '
            str_el = str_el.strip()
            if str_el in collocations_by_freq_dict[n_of_words]:
                overall_colloc_json[n_of_words][str_el] = collocations_by_freq_dict[n_of_words][str_el] 
            else:
                overall_colloc_json[n_of_words][str_el] = colloc.iloc[index][freq_name]
            """
            if str_el in overall_colloc_json[n_of_words]:
                overall_colloc_json[n_of_words][str_el] *= int(colloc.iloc[index][freq_name])
            else:
                overall_colloc_json[n_of_words][str_el] = int(colloc.iloc[index][freq_name])
            """
            #print(str_el)
        #print("========")
    return overall_colloc_json
#ngramms_list = [bigramFreqTable_clean, trigramFreqTable_clean, quadgram_freq_clean, bigramPMITable, trigramPMITable, quadragramPMITable]
ngramms_list = [bigramPMITable, trigramPMITable, quadragramPMITable, bigramChiTable, trigramChiTable,filtered_bi_clean,filtered_tri_clean]

log_collocations_vs_freq = get_united_numeric_colloc_base(ngramms_list,freq_colloc_dict)

In [ ]:
with open ("smart_colloc_freq.json","w") as f:
    json.dump(log_collocations_vs_freq, f, indent = 4, ensure_ascii = False)

In [4]:
import json

In [5]:
with open ("smart_colloc_freq.json","r") as f:
    coll = json.load(f)

In [7]:
coll['3']

{'РґРѕР±С‹С‡Р° РїРѕР»РµР·РЅС‹Р№ РёСЃРєРѕРїР°РµРјРѕРµ': 5,
 'СѓРєСЂР°РёРЅР° РїРµС‚СЂ РїРѕСЂРѕС€РµРЅРєРѕ': 5,
 'РѕСЂСѓР¶РёРµ РјР°СЃСЃРѕРІС‹Р№ РїРѕСЂР°Р¶РµРЅРёРµ': 8,
 'СЂР°РґРёРѕСЃС‚Р°РЅС†РёСЏ СЌС…Рѕ РјРѕСЃРєРІР°': 7,
 'С„РёРЅР°РЅСЃС‹ Р°Р»РµРєСЃРµР№ РєСѓРґСЂРёРЅ': 5,
 'РєРѕРЅС‚СЂРѕР»СЊРЅС‹Р№ РїР°РєРµС‚ Р°РєС†РёСЏ': 5,
 'РёР·РґР°С‚РµР»СЊСЃС‚РІРѕ СЃРµРјСЊ РґРµРЅСЊ': 6,
 'Р±СЋСЂРѕ РєСЂРµРґРёС‚РЅС‹Р№ РёСЃС‚РѕСЂРёСЏ': 10,
 'РІРѕР·Р±СѓР¶РґР°С‚СЊ СѓРіРѕР»РѕРІРЅС‹Р№ РґРµР»Рѕ': 13,
 'Р±РµР»РѕСЂСѓСЃСЃРёСЏ Р°Р»РµРєСЃР°РЅРґСЂ Р»СѓРєР°С€РµРЅРєРѕ': 7,
 'СЃС€Р° РґР¶РѕСЂРґР¶ Р±СѓС€': 5,
 'С†РµРЅС‚СЂР°Р»СЊРЅС‹Р№ РёР·Р±РёСЂР°С‚РµР»СЊРЅС‹Р№ РєРѕРјРёСЃСЃРёСЏ': 5,
 'СЃРѕРѕР±С‰Р°С‚СЊ associated press': 7,
 'РґРѕ СЃРµР№ РїРѕСЂР°': 39,
 'РІРµСЂС…РЅРёР№ РїР°Р»Р°С‚Р° РїР°СЂР»Р°РјРµРЅС‚': 5,
 'РѕСЂРіР°РЅ РјРµСЃС‚РЅС‹Р№ СЃР°РјРѕСѓРїСЂР°РІР»РµРЅРёРµ': 6,
 'СЃРІРѕР±РѕРґРЅС‹Р№ СЌРєРѕРЅРѕРјРёС‡РµСЃРєРёР№ Р·РѕРЅР°': 7,
 'РІС‹СЃС€РёР№ Р°СЂР±РёС‚СЂР°Р¶РЅС‹Р№ СЃСѓРґ': 5,
 'СЃС‚СЂРѕРёС‚РµР»СЊСЃС‚РІРѕ СЃРѕСЋР·РЅС‹Р№ РіРѕСЃСѓ

Exception: Cannot load UDPipe model from file 'english-partut-ud-2.0-170801.udpipe'